# 2. Obtención de datos

## MalagaHouse Scrapping

*Trabajo Fin de Máster FP en IA y Big Data realizado por Miguel Gámez Ruiz y Sergio Toscano Díaz*

In [ ]:
from bs4 import BeautifulSoup
import requests
from csv import writer
import re

In [ ]:
links = set()  # utilizamos un conjunto en lugar de una lista

for p in range(1,74):
    if p == 1:
        #urls = "https://www.pisos.com/venta/pisos-malaga_capital_centro/"
        urls = "https://www.pisos.com/venta/pisos-malaga_capital_zona_urbana/"
    else:
        #urls = "https://www.pisos.com/venta/pisos-malaga_capital_centro/" + str(p) + '/'
        urls = "https://www.pisos.com/venta/pisos-malaga_capital_zona_urbana/" + str(p) + '/'

    page = requests.get(urls)

    soup = BeautifulSoup(page.content, 'html.parser')
    lonks = []

    for seccion in soup.findAll("a", {"class": "ad-preview__title"}):
        lonk = seccion.get("href")
        if "/comprar/" in lonk:
            link1 = "https://www.pisos.com" + lonk
            lonks.append(link1)

    links.update(lonks)  # añadimos los enlaces de la página actual al conjunto de enlaces

links = list(links)  # convertimos el conjunto en una lista
print(links)


['https://www.pisos.com/comprar/casa-pedregalejo29017-24251382855_109300/', 'https://www.pisos.com/comprar/piso-nueva_malaga_miraflores29010-28409178260_100500/', 'https://www.pisos.com/comprar/piso-las_flores_parque_del_sur29014-92555486491_500165/', 'https://www.pisos.com/comprar/piso-el_torcal_nuevo_san_andres_dos_hermanas29003-28361707830_101800/', 'https://www.pisos.com/comprar/piso-el_higueral_palacio_de_deportes_martin_carpena2900-28335754018_100500/', 'https://www.pisos.com/comprar/piso-paseo_maritimo_oeste_pacifico29002-29185127594_512694/', 'https://www.pisos.com/comprar/chalet-el_ejido_cristo_de_la_epidemia_olletas_san_miguel2-30033395047_100500/', 'https://www.pisos.com/comprar/piso-camino_de_suarez29011-31682143877_107700/', 'https://www.pisos.com/comprar/piso-el_torcal_nuevo_san_andres_dos_hermanas29003-27565570494_515048/', 'https://www.pisos.com/comprar/piso-camino_de_suarez29010-30927856701_108300/', 'https://www.pisos.com/comprar/piso-teatinos_cortijo_alto29006-283912

In [ ]:
with open('housing.csv', 'w', encoding='utf8', newline='') as f:
  thewriter = writer(f)
  header = ['Ciudad', 'Precio', 'Superficie', 'Habitaciones', 'Baños', 'Garaje',
          'Trastero', 'Ascensor', 'Terraza', 'Amueblado', 'Chimenea', 'Piscina',
          'Jardín', 'Descripción']
  thewriter.writerow(header)
  
  for urls in links:
    page = requests.get(urls)
    soup = BeautifulSoup(page.content, 'html.parser')

    datosgenericos = soup.findAll("div", {"class": "basicdata-item"})
    datosbasicos = soup.findAll("li", {"class": "charblock-element more-padding"})
    equipamientos = soup.findAll("li", {"class": "charblock-element element-with-bullet"})
    print(urls)
    sup = ""
    hab = ""
    ban = ""
    garaje = 0
    trastero = 0
    ascensor = 0
    terraza = 0
    amueblado = 0
    chimenea = 0
    piscina = 0
    jardin = 0
    ciudad_casa = ""
  
    precio = soup.find('span', {"class": "h1 jsPrecioH1"}).text.strip()
    precio = re.sub('[.€]', '', precio).strip()

    todosdatosbasicos = []
    todosequipamientos = []

    for datobasico in datosbasicos:
      datobasico = datobasico.text
      todosdatosbasicos.append(datobasico)

    for i in todosdatosbasicos:
      if "Superficie construida" in i:
        sup = i
      if "Habitaciones" in i:
        hab = i
      if "Baños" in i and ":"in i and not "Interior" in i:
        ban = i

    numeros = re.compile("(\d+)")
    if sup != '':
      superficie = [int(superficie) for superficie in str.split(sup) if superficie.isdigit()]
      if len(superficie) > 0:
        superficie = superficie[0]
      else:
        superficie = 0
    if hab != '':
      habitaciones = [int(habitaciones) for habitaciones in str.split(hab) if habitaciones.isdigit()]
      habitaciones = habitaciones[0]
    if ban != '':
        banyos = int(re.search('\d+', ban).group())

    for equipamiento in equipamientos:
      equipamiento = equipamiento.text.strip()
      todosequipamientos.append(equipamiento)

    for i in todosequipamientos:
      if "Garaje" in i:
        garaje = 1
      elif "Trastero" in i:
        trastero = 1
      elif "Ascensor" in i:
        ascensor = 1
      elif "Terraza" in i:
        terraza = 1
      elif "Amueblado" in i:
        amueblado = 1
      elif "Chimenea" in i:
        chimenea = 1
      elif "Piscina" in i:
        piscina = 1
      elif "Jardín" in i:
        jardin = 1

    descripcion = soup.find('div', {"class": "description-container description-body"}).text.strip()


    a_centro = ['Capuchinos-La Goleta-El Molinillo-Segalerva', 'La Trinidad-Mármoles-Perchel Norte',
                'Perchel Sur-Plaza de Toros Vieja', 'Ensanche Centro-Puerto', 'Centro Histórico-La Merced', 
                'El Ejido-Cristo de la Epidemia-Olletas-San Miguel', 'La Victoria-Conde Ureña',
                'La Rosaleda-La Roca', 'La Malagueta-Monte Sancha', 'Pinares de Olletas-La Manía-Seminario']

    a_zonas = ['Bailén-Miraflores', 'Campanillas', 'Carretera de Cádiz', 'Centro', 'Churriana', 
                  'Ciudad Jardín', 'Cruz de Humilladero', 'Este', 'La Rosaleda-La Roca', 'Puerto de la Torre', 
                  'Teatinos-Universidad']

    localizacion = soup.find('h2', {"class": "position"}).text.strip()


    #for zona in a_centro:
    #  if zona in localizacion:
    #    zona_casa = zona

    for zona in a_zonas:
      if zona in localizacion:
        zona_casa = zona

    info = [zona_casa, precio, superficie, habitaciones, banyos, garaje,
            trastero, ascensor, terraza, amueblado, chimenea, piscina, jardin,
            descripcion]
    thewriter.writerow(info)

https://www.pisos.com/comprar/casa_adosada-bellavista_el_morlaco29016-28379222854_516020/
https://www.pisos.com/comprar/piso-cruz_de_humilladero_carranque29006-31670834406_520345/
https://www.pisos.com/comprar/piso-cruz_de_humilladero_carranque29007-29188507872_100500/
https://www.pisos.com/comprar/piso-el_ejido_cristo_de_la_epidemia_olletas_san_miguel2-29244576219_100500/
https://www.pisos.com/comprar/piso-camino_de_suarez29009-4252913465_107000/
https://www.pisos.com/comprar/piso-arroyo_de_los_angeles_la_roca29009-2797014009_109700/
https://www.pisos.com/comprar/finca_rustica-el_candado_camino_de_olias_camino_de_los_montes290-20023154343_103200/
https://www.pisos.com/comprar/casa-este_el_palo29018-25841053691_515048/
https://www.pisos.com/comprar/piso-la_virreina_jardin_botanico_la_rosaleda29011-29236337105_100200/
https://www.pisos.com/comprar/piso-camino_de_suarez29011-30857502728_107100/
https://www.pisos.com/comprar/chalet-este_el_palo29018-31679971055_109300/
https://www.pisos.c